In [1]:
import os
import pandas as pd
import glob
import pandas as pd
import glob
import numpy as np
import pandas as pd
import glob
# import win32com.client as win32
import jaydebeapi
import re
# from const import QUERY_CUSTOMERINFO,CONNECT_ORACLE
import sys
import os



In [2]:
# Define the input folder
input_folder = "./input/assign_input/"

# Get a list of files in the folder (xlsx or csv)
files = glob.glob(os.path.join(input_folder, "*.xlsx")) + glob.glob(os.path.join(input_folder, "*.csv"))
print(files)
# Read only the first file found
if files:
    file_path = files[0]
    # print(f"Reading file: {file_path}")
    
    if file_path.endswith(".xlsx"):
        assign = pd.read_excel(file_path,sheet_name="Data", dtype={'Mobile No.': str})
    elif file_path.endswith(".csv"):
        assign = pd.read_csv(file_path,sheet_name="Data", dtype={'Mobile No.': str})
    
    # print(assign.head())  # Display first few rows
else:
    print("No file found in the folder.")

['./input/assign_input\\Template SMS SF+ X_DAY Unsave (Collector ติด Covid) Base.xlsx']


In [3]:
def clean_column_names(df):
    df.columns = df.columns.str.lower()  
    df.columns = df.columns.str.replace(r'\.', '', regex=True)
    df.columns = df.columns.str.replace(r' ', '_', regex=True)  
    df.columns = df.columns.str.replace('customer_id_no|customer_no|customer_id|national_id', 'customer_no', regex=True) 
    df.columns = df.columns.str.replace(r'loan_no', 'contract_no', regex=True)
    # df.columns = df.columns.str.replace(r'mobile.*', 'mobile_no', regex=True)
    df.columns = df.columns.str.replace(r'customer_name/surname\(thai\)', 'customer_name', regex=True)  
    return df

In [4]:
assign

,No,Loan No.,Customer ID No,Customer Name/Surname(Thai),Product code,Overdue Cnt(Morning),Mobile No.,Status,Result_TD,Overdue Sum amt TODAY,4 Digit,Len,Collector,sms_type
0,1,7250270104814187,2320900048950,สุพิน บุญไทย,A701:SF Loan,X_Day,0655897042,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
1,2,7240770103411425,1520100123761,กวินตา สุราฤทธิ์,A701:SF Loan,X_Day,0930248653,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
2,3,7250170104669107,1319900790923,นราดร จีนกูล,A701:SF Loan,X_Day,0826046951,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
3,4,7241070104062890,1549900139899,จอมขวัญ พละบุรี,A701:SF Loan,X_Day,0630193806,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
4,5,7240870103435505,1469900500312,จักรพันธ์ ยุบลพันธ์,A701:SF Loan,X_Day,0830765568,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,7240270102091473,1100701547011,จริยา จันทวิลา,A701:SF Loan,X_Day,0808371796,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
316,317,7240770103292127,1530300063192,ปิยธิดา ลิตตะสาร,A701:SF Loan,X_Day,0652408929,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
317,318,7240770103301835,3849800093475,นิตยา ชูช่วง,A701:SF Loan,X_Day,0946230968,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
318,319,7241070104009274,1101800510265,สิริพร ศรีสกุล,A701:SF Loan,X_Day,0918047912,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave


In [5]:
assign_delay = clean_column_names(assign)

In [6]:
assign_delay

,no,contract_no,customer_no,customer_name,product_code,overdue_cnt(morning),mobile_no,status,result_td,overdue_sum_amt__today,4_digit,len,collector,sms_type
0,1,7250270104814187,2320900048950,สุพิน บุญไทย,A701:SF Loan,X_Day,0655897042,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
1,2,7240770103411425,1520100123761,กวินตา สุราฤทธิ์,A701:SF Loan,X_Day,0930248653,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
2,3,7250170104669107,1319900790923,นราดร จีนกูล,A701:SF Loan,X_Day,0826046951,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
3,4,7241070104062890,1549900139899,จอมขวัญ พละบุรี,A701:SF Loan,X_Day,0630193806,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
4,5,7240870103435505,1469900500312,จักรพันธ์ ยุบลพันธ์,A701:SF Loan,X_Day,0830765568,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,7240270102091473,1100701547011,จริยา จันทวิลา,A701:SF Loan,X_Day,0808371796,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
316,317,7240770103292127,1530300063192,ปิยธิดา ลิตตะสาร,A701:SF Loan,X_Day,0652408929,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
317,318,7240770103301835,3849800093475,นิตยา ชูช่วง,A701:SF Loan,X_Day,0946230968,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave
318,319,7241070104009274,1101800510265,สิริพร ศรีสกุล,A701:SF Loan,X_Day,0918047912,UNSAVE,NaN,NaN,NaN,NaN,Kunlada Thabthimthong,SF+ Unsave


In [14]:
class CONNECT_ORACLE:
    jclassname = "oracle.jdbc.OracleDriver"
    url = "jdbc:oracle:thin:@10.1.41.100:1622/JFDWH"
    # driver_args = ["natdilok","Kbjparn#3009"] #user p parn
    jars = ["supat", "SU#pass005"]
    lib = "ojdbc17.jar"


In [15]:
class CONNECT_TIBERO:
    DB = "com.tmax.tibero.jdbc.TbDriver"
    PORT = "jdbc:tibero:thin:@192.169.10.51:18629:DSTFCC"
    # USER = ["natdilok","Kbjparn#3009"] #user p parn
    USER = ["supat", "coll_sp@2025"]
    CNN = "tibero6-jdbc.jar"

In [18]:
conn = jaydebeapi.connect(
    CONNECT_ORACLE.jclassname,
    CONNECT_ORACLE.url,    # adjust your URL
    CONNECT_ORACLE.jars,
    CONNECT_ORACLE.lib        # full absolute path
)



conn_ti = jaydebeapi.connect(
        CONNECT_TIBERO.DB,
         CONNECT_TIBERO.PORT,
         CONNECT_TIBERO.USER,
         "./tibero6-jdbc.jar",
        )


TypeError: Class com.tmax.tibero.jdbc.TbDriver is not found

In [9]:
customer = pd.read_sql('''WITH data_contract AS (SELECT cid.AS_OF_DATE , cid.CONTRACT_NO AS CONTRACT_NO_val, cid.NATIONAL_ID AS NATIONAL_ID ,cid.CHANNEL ,cid.MONTHLY_INST_AMT ,cid.FIRST_DUE_DATE FROM jfdwh.CONTRACT_INFO_DAILY cid 
WHERE AS_OF_DATE =  (SELECT MAX(AS_OF_DATE) FROM JFDWH.CONTRACT_INFO_DAILY)-1
)
SELECT dc.*,c.NAME AS NAME_val, c.SURNAME  AS SURNAME_val , 
c.MOBILE_PHONE_NO AS MOBILE_PHONE_NO_val ,c.RESIDENT_ADDRESS  FROM JFDWH.data_contract dc
LEFT JOIN JFDWH.CUSTOMER c ON dc.NATIONAL_ID  =c.NATIONAL_ID_NO''', conn) 

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_18216\949800460.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer = pd.read_sql('''WITH data_contract AS (SELECT cid.AS_OF_DATE , cid.CONTRACT_NO AS CONTRACT_NO_val, cid.NATIONAL_ID AS NATIONAL_ID ,cid.CHANNEL ,cid.MONTHLY_INST_AMT ,cid.FIRST_DUE_DATE FROM jfdwh.CONTRACT_INFO_DAILY cid


In [11]:
customer

,AS_OF_DATE,CONTRACT_NO_VAL,NATIONAL_ID,CHANNEL,MONTHLY_INST_AMT,FIRST_DUE_DATE,NAME_VAL,SURNAME_VAL,MOBILE_PHONE_NO_VAL,RESIDENT_ADDRESS
0,2025-04-14 00:00:00,7231270101753545,1101501195619,None,0.00,2024-01-01 00:00:00,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
1,2025-04-14 00:00:00,7231270101852486,1101501195619,None,0.00,2024-02-01 00:00:00,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
2,2025-04-14 00:00:00,7231270101753621,1110200028445,None,0.00,2024-01-01 00:00:00,จิตรดาวัลย์,พุ่มศิริ,0955206058,24 หมู่ที่ 5 ต. คลองสวน อ. บางบ่อ จ. สมุทรปร...
3,2025-04-14 00:00:00,7231270101753712,1329900063061,None,441.71,2024-01-01 00:00:00,ลัดดารัตน์,เงางาม,0826861321,61 หมู่ที่ 9 ต. บึง อ. เขวาสินรินทร์ จ. สุริ...
4,2025-04-14 00:00:00,7231270101834614,5570700109443,None,736.32,2024-02-01 00:00:00,วันเพ็ญ,กันหมู่ร้าย,0902905574,10/115 ซอยเฉลิมพระเกียรติ ร.๙ ซอย30แยก14-1 แ...
...,...,...,...,...,...,...,...,...,...,...
5564819,2025-04-14 00:00:00,7240170101896646,1961100126538,None,0.00,2024-02-01 00:00:00,อิสมาแอ,กาเดย์,0806264380,68 หมู่ที่ 2 ต. กาวะ อ. สุไหงปาดี จ. นราธิวาส
5564820,2025-04-14 00:00:00,7231270101828629,3810100098123,None,0.00,2024-02-01 00:00:00,ปรีดา,ชูศรี,0623824495,142 หมู่ที่ 1 ต. เขาทอง อ. เมืองกระบี่ จ. กร...
5564821,2025-04-14 00:00:00,7231270101827769,3810100098123,None,455.48,2024-02-01 00:00:00,ปรีดา,ชูศรี,0623824495,142 หมู่ที่ 1 ต. เขาทอง อ. เมืองกระบี่ จ. กร...
5564822,2025-04-14 00:00:00,7240170101900587,1200600087065,None,0.00,2024-02-01 00:00:00,วาริชา,อินทรีย์,0618841941,87/1 หมู่ที่ 1 ต. หนองปรือ อ. พนัสนิคม จ. ชล...


In [ ]:
# customer = pd.read_csv('./input/customer_input/customerdata.csv', dtype={'MOBILE_PHONE_NO_VAL': str, 
#                                                             'NATIONAL_ID': str, 
#                                                             'CONTRACT_NO_VAL': str})

In [ ]:
# customer

,as_of_date,contract_no_val,customer_no,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address
0,2025-04-15 00:00:00.000,7231270101753545,1101501195619,NaN,0.00,2024-01-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
1,2025-04-15 00:00:00.000,7231270101852486,1101501195619,NaN,0.00,2024-02-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
2,2025-04-15 00:00:00.000,7231270101753621,1110200028445,NaN,0.00,2024-01-01 00:00:00.000,จิตรดาวัลย์,พุ่มศิริ,0955206058,24 หมู่ที่ 5 ต. คลองสวน อ. บางบ่อ จ. สมุทรปร...
3,2025-04-15 00:00:00.000,7231270101753712,1329900063061,NaN,441.71,2024-01-01 00:00:00.000,ลัดดารัตน์,เงางาม,0826861321,61 หมู่ที่ 9 ต. บึง อ. เขวาสินรินทร์ จ. สุริ...
4,2025-04-15 00:00:00.000,7231270101834614,5570700109443,NaN,736.32,2024-02-01 00:00:00.000,วันเพ็ญ,กันหมู่ร้าย,0902905574,10/115 ซอยเฉลิมพระเกียรติ ร.๙ ซอย30แยก14-1 แ...
...,...,...,...,...,...,...,...,...,...,...
5570619,2025-04-15 00:00:00.000,7231270101796392,1650900066264,NaN,0.00,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570620,2025-04-15 00:00:00.000,7231270101850564,1650900066264,NaN,819.20,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570621,2025-04-15 00:00:00.000,7231270101712732,1779900233711,NaN,1672.68,2024-01-01 00:00:00.000,เพชรรัตน์,ศรีเมือง,0800752498,20/7 หมู่ที่ 7 ต. กุยบุรี อ. กุยบุรี จ. ประจ...
5570622,2025-04-15 00:00:00.000,7250470105173060,1801300154455,NaN,187.66,2025-06-01 00:00:00.000,NaN,NaN,NaN,NaN


In [11]:
class CONNECT_TIBERO:
    DB = "com.tmax.tibero.jdbc.TbDriver"
    PORT = "jdbc:tibero:thin:@192.169.10.51:18629:DSTFCC"
    # USER = ["natdilok","Kbjparn#3009"] #user p parn
    USER = ["supat", "coll_sp@2025"]
    CNN = "tibero6-jdbc.jar"

In [12]:
conn_ti = jaydebeapi.connect(
        CONNECT_TIBERO.DB,
         CONNECT_TIBERO.PORT,
         CONNECT_TIBERO.USER,
         CONNECT_TIBERO.CNN,
        )
cur = conn_ti.cursor()  

sms_type = pd.read_sql(f'''
    SELECT * FROM SIREETRON.REF_SMS_WORDING 
    ''', conn_ti) 
conn_ti.close()

TypeError: Class com.tmax.tibero.jdbc.TbDriver is not found

In [182]:
sms_type

,ID,SMS_TYPE,SMS_WORDING
0,1,C4C,สินเชื่อรถยนต์แคชจอยของคุณลงท้าย {4digit} ครบก...
1,2,Unsave,สินเชื่อลงท้าย {4digit} ของท่านมียอดค้างชำระ โ...
2,3,In-House Unsave,สินเชื่อแคชจอยของท่านเกินกำหนดชำระ เพื่อหลีกเล...
3,4,SF+ Unsave,สินเชื่อของท่านค้างชำระ {totalunpaidbalance} บ...
4,5,SF+ non assign,สินเชื่อ Samsung บัญชีลงท้าย {4digit} เกินกำหน...
5,6,Delayed Calling,สินเชื่อแคชจอยของท่านเกินกำหนดชำระ เพื่อหลีกเล...
6,7,Offer TDR (old),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
7,8,TDR (GPD/Normal),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
8,9,TDR (SPD only),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
9,10,Offer Settlement,KBJ Capital ขอเสนอส่วนลดพิเศษสูงสุด 30% สำหรับ...


In [183]:
assign_delay

,no,contract_no,customer_no,customer_name,product_code,overdue_cnt(morning),mobile_no,status,result_td,overdue_sum_amt__today,4_digit,len,collector,sms_type
0,1,7250270104814187,2320900048950,สุพิน บุญไทย,A701:SF Loan,X_Day,0655897042,UNSAVE,NaN,2357.34,4187,10,Kunlada Thabthimthong,SF+ Unsave
1,2,7240770103411425,1520100123761,กวินตา สุราฤทธิ์,A701:SF Loan,X_Day,0930248653,UNSAVE,NaN,2654.54,1425,10,Kunlada Thabthimthong,SF+ Unsave
2,3,7250170104669107,1319900790923,นราดร จีนกูล,A701:SF Loan,X_Day,0826046951,UNSAVE,NaN,1777.95,9107,10,Kunlada Thabthimthong,SF+ Unsave
3,4,7241070104062890,1549900139899,จอมขวัญ พละบุรี,A701:SF Loan,X_Day,0630193806,UNSAVE,NaN,1936.23,2890,10,Kunlada Thabthimthong,SF+ Unsave
4,5,7240870103435505,1469900500312,จักรพันธ์ ยุบลพันธ์,A701:SF Loan,X_Day,0830765568,UNSAVE,NaN,2205.70,5505,10,Kunlada Thabthimthong,SF+ Unsave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,7240270102091473,1100701547011,จริยา จันทวิลา,A701:SF Loan,X_Day,0808371796,UNSAVE,NaN,841.70,1473,10,Kunlada Thabthimthong,SF+ Unsave
316,317,7240770103292127,1530300063192,ปิยธิดา ลิตตะสาร,A701:SF Loan,X_Day,0652408929,UNSAVE,NaN,736.18,2127,10,Kunlada Thabthimthong,SF+ Unsave
317,318,7240770103301835,3849800093475,นิตยา ชูช่วง,A701:SF Loan,X_Day,0946230968,UNSAVE,NaN,736.18,1835,10,Kunlada Thabthimthong,SF+ Unsave
318,319,7241070104009274,1101800510265,สิริพร ศรีสกุล,A701:SF Loan,X_Day,0918047912,UNSAVE,NaN,460.08,9274,10,Kunlada Thabthimthong,SF+ Unsave


In [184]:
customer

,as_of_date,contract_no_val,customer_no,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address
0,2025-04-15 00:00:00.000,7231270101753545,1101501195619,NaN,0.00,2024-01-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
1,2025-04-15 00:00:00.000,7231270101852486,1101501195619,NaN,0.00,2024-02-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
2,2025-04-15 00:00:00.000,7231270101753621,1110200028445,NaN,0.00,2024-01-01 00:00:00.000,จิตรดาวัลย์,พุ่มศิริ,0955206058,24 หมู่ที่ 5 ต. คลองสวน อ. บางบ่อ จ. สมุทรปร...
3,2025-04-15 00:00:00.000,7231270101753712,1329900063061,NaN,441.71,2024-01-01 00:00:00.000,ลัดดารัตน์,เงางาม,0826861321,61 หมู่ที่ 9 ต. บึง อ. เขวาสินรินทร์ จ. สุริ...
4,2025-04-15 00:00:00.000,7231270101834614,5570700109443,NaN,736.32,2024-02-01 00:00:00.000,วันเพ็ญ,กันหมู่ร้าย,0902905574,10/115 ซอยเฉลิมพระเกียรติ ร.๙ ซอย30แยก14-1 แ...
...,...,...,...,...,...,...,...,...,...,...
5570619,2025-04-15 00:00:00.000,7231270101796392,1650900066264,NaN,0.00,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570620,2025-04-15 00:00:00.000,7231270101850564,1650900066264,NaN,819.20,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570621,2025-04-15 00:00:00.000,7231270101712732,1779900233711,NaN,1672.68,2024-01-01 00:00:00.000,เพชรรัตน์,ศรีเมือง,0800752498,20/7 หมู่ที่ 7 ต. กุยบุรี อ. กุยบุรี จ. ประจ...
5570622,2025-04-15 00:00:00.000,7250470105173060,1801300154455,NaN,187.66,2025-06-01 00:00:00.000,NaN,NaN,NaN,NaN


In [185]:
customer.columns = customer.columns.str.lower()  
customer

,as_of_date,contract_no_val,customer_no,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address
0,2025-04-15 00:00:00.000,7231270101753545,1101501195619,NaN,0.00,2024-01-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
1,2025-04-15 00:00:00.000,7231270101852486,1101501195619,NaN,0.00,2024-02-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
2,2025-04-15 00:00:00.000,7231270101753621,1110200028445,NaN,0.00,2024-01-01 00:00:00.000,จิตรดาวัลย์,พุ่มศิริ,0955206058,24 หมู่ที่ 5 ต. คลองสวน อ. บางบ่อ จ. สมุทรปร...
3,2025-04-15 00:00:00.000,7231270101753712,1329900063061,NaN,441.71,2024-01-01 00:00:00.000,ลัดดารัตน์,เงางาม,0826861321,61 หมู่ที่ 9 ต. บึง อ. เขวาสินรินทร์ จ. สุริ...
4,2025-04-15 00:00:00.000,7231270101834614,5570700109443,NaN,736.32,2024-02-01 00:00:00.000,วันเพ็ญ,กันหมู่ร้าย,0902905574,10/115 ซอยเฉลิมพระเกียรติ ร.๙ ซอย30แยก14-1 แ...
...,...,...,...,...,...,...,...,...,...,...
5570619,2025-04-15 00:00:00.000,7231270101796392,1650900066264,NaN,0.00,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570620,2025-04-15 00:00:00.000,7231270101850564,1650900066264,NaN,819.20,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570621,2025-04-15 00:00:00.000,7231270101712732,1779900233711,NaN,1672.68,2024-01-01 00:00:00.000,เพชรรัตน์,ศรีเมือง,0800752498,20/7 หมู่ที่ 7 ต. กุยบุรี อ. กุยบุรี จ. ประจ...
5570622,2025-04-15 00:00:00.000,7250470105173060,1801300154455,NaN,187.66,2025-06-01 00:00:00.000,NaN,NaN,NaN,NaN


In [186]:
assign_delay

,no,contract_no,customer_no,customer_name,product_code,overdue_cnt(morning),mobile_no,status,result_td,overdue_sum_amt__today,4_digit,len,collector,sms_type
0,1,7250270104814187,2320900048950,สุพิน บุญไทย,A701:SF Loan,X_Day,0655897042,UNSAVE,NaN,2357.34,4187,10,Kunlada Thabthimthong,SF+ Unsave
1,2,7240770103411425,1520100123761,กวินตา สุราฤทธิ์,A701:SF Loan,X_Day,0930248653,UNSAVE,NaN,2654.54,1425,10,Kunlada Thabthimthong,SF+ Unsave
2,3,7250170104669107,1319900790923,นราดร จีนกูล,A701:SF Loan,X_Day,0826046951,UNSAVE,NaN,1777.95,9107,10,Kunlada Thabthimthong,SF+ Unsave
3,4,7241070104062890,1549900139899,จอมขวัญ พละบุรี,A701:SF Loan,X_Day,0630193806,UNSAVE,NaN,1936.23,2890,10,Kunlada Thabthimthong,SF+ Unsave
4,5,7240870103435505,1469900500312,จักรพันธ์ ยุบลพันธ์,A701:SF Loan,X_Day,0830765568,UNSAVE,NaN,2205.70,5505,10,Kunlada Thabthimthong,SF+ Unsave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,7240270102091473,1100701547011,จริยา จันทวิลา,A701:SF Loan,X_Day,0808371796,UNSAVE,NaN,841.70,1473,10,Kunlada Thabthimthong,SF+ Unsave
316,317,7240770103292127,1530300063192,ปิยธิดา ลิตตะสาร,A701:SF Loan,X_Day,0652408929,UNSAVE,NaN,736.18,2127,10,Kunlada Thabthimthong,SF+ Unsave
317,318,7240770103301835,3849800093475,นิตยา ชูช่วง,A701:SF Loan,X_Day,0946230968,UNSAVE,NaN,736.18,1835,10,Kunlada Thabthimthong,SF+ Unsave
318,319,7241070104009274,1101800510265,สิริพร ศรีสกุล,A701:SF Loan,X_Day,0918047912,UNSAVE,NaN,460.08,9274,10,Kunlada Thabthimthong,SF+ Unsave


In [187]:
customer

,as_of_date,contract_no_val,customer_no,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address
0,2025-04-15 00:00:00.000,7231270101753545,1101501195619,NaN,0.00,2024-01-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
1,2025-04-15 00:00:00.000,7231270101852486,1101501195619,NaN,0.00,2024-02-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
2,2025-04-15 00:00:00.000,7231270101753621,1110200028445,NaN,0.00,2024-01-01 00:00:00.000,จิตรดาวัลย์,พุ่มศิริ,0955206058,24 หมู่ที่ 5 ต. คลองสวน อ. บางบ่อ จ. สมุทรปร...
3,2025-04-15 00:00:00.000,7231270101753712,1329900063061,NaN,441.71,2024-01-01 00:00:00.000,ลัดดารัตน์,เงางาม,0826861321,61 หมู่ที่ 9 ต. บึง อ. เขวาสินรินทร์ จ. สุริ...
4,2025-04-15 00:00:00.000,7231270101834614,5570700109443,NaN,736.32,2024-02-01 00:00:00.000,วันเพ็ญ,กันหมู่ร้าย,0902905574,10/115 ซอยเฉลิมพระเกียรติ ร.๙ ซอย30แยก14-1 แ...
...,...,...,...,...,...,...,...,...,...,...
5570619,2025-04-15 00:00:00.000,7231270101796392,1650900066264,NaN,0.00,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570620,2025-04-15 00:00:00.000,7231270101850564,1650900066264,NaN,819.20,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570621,2025-04-15 00:00:00.000,7231270101712732,1779900233711,NaN,1672.68,2024-01-01 00:00:00.000,เพชรรัตน์,ศรีเมือง,0800752498,20/7 หมู่ที่ 7 ต. กุยบุรี อ. กุยบุรี จ. ประจ...
5570622,2025-04-15 00:00:00.000,7250470105173060,1801300154455,NaN,187.66,2025-06-01 00:00:00.000,NaN,NaN,NaN,NaN


In [188]:
customer = clean_column_names(customer)

In [189]:
customer


,as_of_date,contract_no_val,customer_no,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address
0,2025-04-15 00:00:00.000,7231270101753545,1101501195619,NaN,0.00,2024-01-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
1,2025-04-15 00:00:00.000,7231270101852486,1101501195619,NaN,0.00,2024-02-01 00:00:00.000,ศลิษา,สีเทียนสุก,0953185583,43/61 หมู่ที่ 1 แขวง จอมทอง เขต จอมทอง กรุงเ...
2,2025-04-15 00:00:00.000,7231270101753621,1110200028445,NaN,0.00,2024-01-01 00:00:00.000,จิตรดาวัลย์,พุ่มศิริ,0955206058,24 หมู่ที่ 5 ต. คลองสวน อ. บางบ่อ จ. สมุทรปร...
3,2025-04-15 00:00:00.000,7231270101753712,1329900063061,NaN,441.71,2024-01-01 00:00:00.000,ลัดดารัตน์,เงางาม,0826861321,61 หมู่ที่ 9 ต. บึง อ. เขวาสินรินทร์ จ. สุริ...
4,2025-04-15 00:00:00.000,7231270101834614,5570700109443,NaN,736.32,2024-02-01 00:00:00.000,วันเพ็ญ,กันหมู่ร้าย,0902905574,10/115 ซอยเฉลิมพระเกียรติ ร.๙ ซอย30แยก14-1 แ...
...,...,...,...,...,...,...,...,...,...,...
5570619,2025-04-15 00:00:00.000,7231270101796392,1650900066264,NaN,0.00,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570620,2025-04-15 00:00:00.000,7231270101850564,1650900066264,NaN,819.20,2024-02-01 00:00:00.000,อติชาติ,เข็มทอง,0868613411,641/1 หมู่ที่ 4 ต. ชมพู อ. เนินมะปราง จ. พิษ...
5570621,2025-04-15 00:00:00.000,7231270101712732,1779900233711,NaN,1672.68,2024-01-01 00:00:00.000,เพชรรัตน์,ศรีเมือง,0800752498,20/7 หมู่ที่ 7 ต. กุยบุรี อ. กุยบุรี จ. ประจ...
5570622,2025-04-15 00:00:00.000,7250470105173060,1801300154455,NaN,187.66,2025-06-01 00:00:00.000,NaN,NaN,NaN,NaN


In [190]:
sms_type = clean_column_names(sms_type)

In [191]:
assign_delay = assign_delay.merge(sms_type,left_on='sms_type',right_on = 'sms_type', how='left')
# assign_delay['customer_no'] = assign_delay['customer_no'].astype(str)
assign_delay['contract_no'] = assign_delay['contract_no'].astype(str)
customer['contract_no_val'] = customer['contract_no_val'].astype(str)
# customer['national_id'] = customer['national_id'].astype(str) 

In [192]:
# assign_delay_merge = assign_delay.merge(customer, left_on=['customer_no', 'contract_no'], right_on=['national_id' ,'contract_no_val'], how='left')
assign_delay_merge = assign_delay.merge(customer, left_on= 'contract_no', right_on='contract_no_val', how='left')


In [193]:
assign_delay_merge.columns

Index(['no', 'contract_no', 'customer_no_x', 'customer_name', 'product_code',
       'overdue_cnt(morning)', 'mobile_no', 'status', 'result_td',
       'overdue_sum_amt__today', '4_digit', 'len', 'collector', 'sms_type',
       'id', 'sms_wording', 'as_of_date', 'contract_no_val', 'customer_no_y',
       'channel', 'monthly_inst_amt', 'first_due_date', 'name_val',
       'surname_val', 'mobile_phone_no_val', 'resident_address'],
      dtype='object')

In [194]:
assign_delay_merge

,no,contract_no,customer_no_x,customer_name,product_code,overdue_cnt(morning),mobile_no,status,result_td,overdue_sum_amt__today,...,as_of_date,contract_no_val,customer_no_y,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address
0,1,7250270104814187,2320900048950,สุพิน บุญไทย,A701:SF Loan,X_Day,0655897042,UNSAVE,NaN,2357.34,...,2025-04-15 00:00:00.000,7250270104814187,2320900048950,NaN,2357.34,2025-04-01 00:00:00.000,สุพิน,บุญไทย,0655897042,42 หมู่ที่ 6 ต. หนองเหล็ก อ. ศีขรภูมิ จ. สุร...
1,2,7240770103411425,1520100123761,กวินตา สุราฤทธิ์,A701:SF Loan,X_Day,0930248653,UNSAVE,NaN,2654.54,...,2025-04-15 00:00:00.000,7240770103411425,1520100123761,NaN,2604.54,2024-09-01 00:00:00.000,กวินตา,สุราฤทธิ์,0930248653,506/46 ริมทางรถไฟสายแปดริ้ว แขวง ถนนพญาไท เข...
2,3,7250170104669107,1319900790923,นราดร จีนกูล,A701:SF Loan,X_Day,0826046951,UNSAVE,NaN,1777.95,...,2025-04-15 00:00:00.000,7250170104669107,1319900790923,NaN,1777.95,2025-03-01 00:00:00.000,นราดร,จีนกูล,0826046951,191 หมู่ที่ 3 ต. นางรอง อ. นางรอง จ. บุรีรัมย์
3,4,7241070104062890,1549900139899,จอมขวัญ พละบุรี,A701:SF Loan,X_Day,0630193806,UNSAVE,NaN,1936.23,...,2025-04-15 00:00:00.000,7241070104062890,1549900139899,NaN,1886.23,2024-12-01 00:00:00.000,จอมขวัญ,พละบุรี,0630193806,14 คำลือ ต. ในเวียง อ. เมืองแพร่ จ. แพร่
4,5,7240870103435505,1469900500312,จักรพันธ์ ยุบลพันธ์,A701:SF Loan,X_Day,0830765568,UNSAVE,NaN,2205.70,...,2025-04-15 00:00:00.000,7240870103435505,1469900500312,NaN,2155.70,2024-09-01 00:00:00.000,จักรพันธ์,ยุบลพันธ์,0830765568,100/9 หมู่ที่ 4 ต. กกตูม อ. ดงหลวง จ. มุกดาหาร
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,7240270102091473,1100701547011,จริยา จันทวิลา,A701:SF Loan,X_Day,0808371796,UNSAVE,NaN,841.70,...,2025-04-15 00:00:00.000,7240270102091473,1100701547011,NaN,741.70,2024-03-01 00:00:00.000,จริยา,จันทวิลา,0808371796,90 หมู่ที่ 17 ต. หนองสาหร่าย อ. ปากช่อง จ. น...
316,317,7240770103292127,1530300063192,ปิยธิดา ลิตตะสาร,A701:SF Loan,X_Day,0652408929,UNSAVE,NaN,736.18,...,2025-04-15 00:00:00.000,7240770103292127,1530300063192,NaN,736.18,2024-09-01 00:00:00.000,ปิยธิดา,ลิตตะสาร,0652408929,79 หมู่ที่ 5 ต. ท่าปลา อ. ท่าปลา จ. อุตรดิตถ์
317,318,7240770103301835,3849800093475,นิตยา ชูช่วง,A701:SF Loan,X_Day,0946230968,UNSAVE,NaN,736.18,...,2025-04-15 00:00:00.000,7240770103301835,3849800093475,NaN,736.18,2024-09-01 00:00:00.000,นิตยา,ชูช่วง,0946230968,60/7 เหมืองแกะ ต. นาสาร อ. บ้านนาสาร จ. สุรา...
318,319,7241070104009274,1101800510265,สิริพร ศรีสกุล,A701:SF Loan,X_Day,0918047912,UNSAVE,NaN,460.08,...,2025-04-15 00:00:00.000,7241070104009274,1101800510265,NaN,460.08,2024-12-01 00:00:00.000,สิริพร,ศรีสกุล,0918047912,235 หมู่ที่ 7 ต. หนองหว้า อ. กุมภวาปี จ. อุด...


In [195]:
assign_delay_merge['mobile_check'] = assign_delay_merge.apply(
    lambda row: True if row['mobile_no'] == row['mobile_phone_no_val'] else row['mobile_phone_no_val'],
    axis=1
)

In [196]:
assign_delay_merge['last4digit_val'] = assign_delay_merge['contract_no'].str[-4:]
assign_delay_merge['sms_wording'] = assign_delay_merge.apply(lambda row: row['sms_wording'].replace("{4digit}", str(row['last4digit_val'])), axis=1)


In [197]:
assign_delay_merge

,no,contract_no,customer_no_x,customer_name,product_code,overdue_cnt(morning),mobile_no,status,result_td,overdue_sum_amt__today,...,customer_no_y,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address,mobile_check,last4digit_val
0,1,7250270104814187,2320900048950,สุพิน บุญไทย,A701:SF Loan,X_Day,0655897042,UNSAVE,NaN,2357.34,...,2320900048950,NaN,2357.34,2025-04-01 00:00:00.000,สุพิน,บุญไทย,0655897042,42 หมู่ที่ 6 ต. หนองเหล็ก อ. ศีขรภูมิ จ. สุร...,True,4187
1,2,7240770103411425,1520100123761,กวินตา สุราฤทธิ์,A701:SF Loan,X_Day,0930248653,UNSAVE,NaN,2654.54,...,1520100123761,NaN,2604.54,2024-09-01 00:00:00.000,กวินตา,สุราฤทธิ์,0930248653,506/46 ริมทางรถไฟสายแปดริ้ว แขวง ถนนพญาไท เข...,True,1425
2,3,7250170104669107,1319900790923,นราดร จีนกูล,A701:SF Loan,X_Day,0826046951,UNSAVE,NaN,1777.95,...,1319900790923,NaN,1777.95,2025-03-01 00:00:00.000,นราดร,จีนกูล,0826046951,191 หมู่ที่ 3 ต. นางรอง อ. นางรอง จ. บุรีรัมย์,True,9107
3,4,7241070104062890,1549900139899,จอมขวัญ พละบุรี,A701:SF Loan,X_Day,0630193806,UNSAVE,NaN,1936.23,...,1549900139899,NaN,1886.23,2024-12-01 00:00:00.000,จอมขวัญ,พละบุรี,0630193806,14 คำลือ ต. ในเวียง อ. เมืองแพร่ จ. แพร่,True,2890
4,5,7240870103435505,1469900500312,จักรพันธ์ ยุบลพันธ์,A701:SF Loan,X_Day,0830765568,UNSAVE,NaN,2205.70,...,1469900500312,NaN,2155.70,2024-09-01 00:00:00.000,จักรพันธ์,ยุบลพันธ์,0830765568,100/9 หมู่ที่ 4 ต. กกตูม อ. ดงหลวง จ. มุกดาหาร,True,5505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,7240270102091473,1100701547011,จริยา จันทวิลา,A701:SF Loan,X_Day,0808371796,UNSAVE,NaN,841.70,...,1100701547011,NaN,741.70,2024-03-01 00:00:00.000,จริยา,จันทวิลา,0808371796,90 หมู่ที่ 17 ต. หนองสาหร่าย อ. ปากช่อง จ. น...,True,1473
316,317,7240770103292127,1530300063192,ปิยธิดา ลิตตะสาร,A701:SF Loan,X_Day,0652408929,UNSAVE,NaN,736.18,...,1530300063192,NaN,736.18,2024-09-01 00:00:00.000,ปิยธิดา,ลิตตะสาร,0652408929,79 หมู่ที่ 5 ต. ท่าปลา อ. ท่าปลา จ. อุตรดิตถ์,True,2127
317,318,7240770103301835,3849800093475,นิตยา ชูช่วง,A701:SF Loan,X_Day,0946230968,UNSAVE,NaN,736.18,...,3849800093475,NaN,736.18,2024-09-01 00:00:00.000,นิตยา,ชูช่วง,0946230968,60/7 เหมืองแกะ ต. นาสาร อ. บ้านนาสาร จ. สุรา...,True,1835
318,319,7241070104009274,1101800510265,สิริพร ศรีสกุล,A701:SF Loan,X_Day,0918047912,UNSAVE,NaN,460.08,...,1101800510265,NaN,460.08,2024-12-01 00:00:00.000,สิริพร,ศรีสกุล,0918047912,235 หมู่ที่ 7 ต. หนองหว้า อ. กุมภวาปี จ. อุด...,True,9274


In [198]:
ar_file = pd.read_csv('./input/ar//Acc_20250416_0830.csv')

In [199]:
ar_file['Loan No'] = ar_file['Loan No'].astype(str)

In [200]:
ar_file.columns

Index(['BaseDate', 'Loan No', 'Product Code', 'Overdue Start Date',
       'OverdueDays_Morning', 'OverdueCnt_Morning', 'OverdueDays',
       'OverdueCnt', 'Loan Amt', 'Loan Balance', 'Outstanding Balance',
       'OverdueSumAmt', 'OverduePrin', 'OverdueInt', 'OverdueCrUsage',
       'OverdueOutputTax', 'OverdueCollectionFee', 'OverdueVAT',
       'OverdueStampTaxAmt', 'OverdueNCBTaxAmt', 'OverdueETCFeeAmt', 'LawCost',
       'LastPaymentDt', 'LastPaymentAmt'],
      dtype='object')

In [201]:
ar_file =ar_file[['Loan No','OverdueSumAmt']]

In [202]:
assign_delay_merge = assign_delay_merge.merge(ar_file,left_on='contract_no',right_on="Loan No",how='left')

In [203]:
assign_delay_merge.columns

Index(['no', 'contract_no', 'customer_no_x', 'customer_name', 'product_code',
       'overdue_cnt(morning)', 'mobile_no', 'status', 'result_td',
       'overdue_sum_amt__today', '4_digit', 'len', 'collector', 'sms_type',
       'id', 'sms_wording', 'as_of_date', 'contract_no_val', 'customer_no_y',
       'channel', 'monthly_inst_amt', 'first_due_date', 'name_val',
       'surname_val', 'mobile_phone_no_val', 'resident_address',
       'mobile_check', 'last4digit_val', 'Loan No', 'OverdueSumAmt'],
      dtype='object')

In [204]:
ar_file[['Loan No','OverdueSumAmt']]

,Loan No,OverdueSumAmt
0,9010221122000949,16750.0
1,9010221122000931,0.0
2,9010221122000922,0.0
3,9010221122000914,0.0
4,9010221122000906,0.0
...,...,...
1436316,1000158090014831,100.0
1436317,1000158090014814,30.0
1436318,1000158090008580,100.0
1436319,1000158090005394,100.0


In [208]:
assign_delay_merge['sms_wording'] = assign_delay_merge.apply(lambda row: row['sms_wording'].replace("{totalunpaidbalance}", str(row['OverdueSumAmt'])), axis=1)

In [209]:
assign_delay_merge.columns

Index(['no', 'contract_no', 'customer_no_x', 'customer_name', 'product_code',
       'overdue_cnt(morning)', 'mobile_no', 'status', 'result_td',
       'overdue_sum_amt__today', '4_digit', 'len', 'collector', 'sms_type',
       'id', 'sms_wording', 'as_of_date', 'contract_no_val', 'customer_no_y',
       'channel', 'monthly_inst_amt', 'first_due_date', 'name_val',
       'surname_val', 'mobile_phone_no_val', 'resident_address',
       'mobile_check', 'last4digit_val', 'Loan No', 'OverdueSumAmt'],
      dtype='object')

In [211]:
assign_delay_merge[['sms_wording']]

,sms_wording
0,สินเชื่อของท่านค้างชำระ 2357.34 บาท เพื่อหลีกเ...
1,สินเชื่อของท่านค้างชำระ 2654.54 บาท เพื่อหลีกเ...
2,สินเชื่อของท่านค้างชำระ 1777.95 บาท เพื่อหลีกเ...
3,สินเชื่อของท่านค้างชำระ 1936.23 บาท เพื่อหลีกเ...
4,สินเชื่อของท่านค้างชำระ 2205.7 บาท เพื่อหลีกเล...
...,...
315,สินเชื่อของท่านค้างชำระ 841.7 บาท เพื่อหลีกเลี...
316,สินเชื่อของท่านค้างชำระ 736.18 บาท เพื่อหลีกเล...
317,สินเชื่อของท่านค้างชำระ 736.18 บาท เพื่อหลีกเล...
318,สินเชื่อของท่านค้างชำระ 460.08 บาท เพื่อหลีกเล...


In [ ]:
from openpyxl import load_workbook
from pandas import ExcelWriter

# with ExcelWriter('./input/assign_input/Template SMS SF+ X_DAY Unsave (Collector ติด Covid) Base.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
#     assign_delay_merge.to_excel(writer, sheet_name='check', index=False)

In [214]:
import shutil

In [215]:
original_path = './input/assign_input/Template SMS SF+ X_DAY Unsave (Collector ติด Covid) Base.xlsx'
backup_path = './output/Template_SMS_SF_Backup.xlsx'
shutil.copyfile(original_path, backup_path)

'./output/Template_SMS_SF_Backup.xlsx'

In [212]:
assign_delay_merge.to_excel('./output/iiii.xlsx',index=False)